In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import re
import matplotlib.pyplot as plt
import random
import json
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error
from torch.nn import MSELoss

# WCZYTYWANIE DANYCH

In [ ]:
def load_timestep_data(file_path):
    df = pd.read_csv(file_path, sep=r'\s+', header=None, names=["bitrate","connection_id"])
    bitrate = df['bitrate'].tolist()
    return bitrate

In [3]:
def custom_sort_key_files(filename):
    match = re.search(r"(\d+)", filename)
    if match:
        main_num = int(match.group(1)) 
        return (main_num)
    return (float('inf'))

def custom_sort_key_dirs(filename):
    match = re.search(r"(\d+)_(\d+)", filename)
    if match:
        main_num = int(match.group(1))  
        sub_num = int(match.group(2))  
        return (main_num, sub_num)
    return (float('inf'), float('inf'))

def get_all_test_data(root_folder, adjacency_matrix):
    subfolders = [f for f in os.listdir(root_folder) if f.startswith("processed_7000_")]
    print(subfolders)
    subfolders.sort(key=custom_sort_key_dirs)
    all_data = []
    for subfolder in subfolders:
        data_folder = os.path.join(root_folder, subfolder)
        print(data_folder)
        graph_data_list = load_data_with_targets(data_folder, adjacency_matrix)

        test_data = graph_data_list[6000:]

        all_data.append(test_data)
    return all_data
        
    
def load_data_with_targets(data_folder, adjacency_matrix):

    graph_data_list = []
    files = sorted(os.listdir(data_folder))
    files.sort(key=custom_sort_key_files)

    bitrate_data = [load_timestep_data(os.path.join(data_folder, file)) for file in files]


    for i in range(len(bitrate_data) - 1):
        bitrate = bitrate_data[i]
        target_bitrate = bitrate_data[i + 1]
        graph_data = create_graph_data_with_targets(bitrate, target_bitrate, adjacency_matrix)
        graph_data_list.append(graph_data)

    return graph_data_list


def create_graph_data_with_targets(bitrate, target_bitrate, adjacency_matrix):
    edge_index = torch.tensor(np.array(np.nonzero(adjacency_matrix)), dtype=torch.long)
    edge_weight = torch.tensor(adjacency_matrix[np.nonzero(adjacency_matrix)], dtype=torch.float)
    x = torch.tensor(bitrate, dtype=torch.float).view(-1, 1)
    y = torch.tensor(target_bitrate, dtype=torch.float).view(-1, 1)
    data = Data(x=x, edge_index=edge_index, edge_weight=edge_weight, y=y)
    return data


In [4]:
def load_adjacency_matrix(file_path):
    adjacency_matrix = np.loadtxt(file_path, delimiter='\t')
    return adjacency_matrix

# MODEL

In [5]:
class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.gru = torch.nn.GRU(hidden_dim, hidden_dim)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data, h):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        out, h = self.gru(x.unsqueeze(0), h)
        out = self.fc(out.squeeze(0))
        return out, h


# TESTOWANIE, METRYKI

In [6]:
def test_model_with_tolerance(model, test_data_list, starting_index=999, tolerance=0.15, criterion=MSELoss()):
    model.eval()
    total_loss = 0
    within_tolerance_count = 0
    total_predictions = 0
    predictions = []
    current_index = 0
    with torch.no_grad():
        for data in test_data_list:
            h = torch.zeros(1, data.x.size(0), model.gru.hidden_size).to(data.x.device)
            out, h = model(data, h)
            loss = criterion(out, data.y)
            total_loss += loss.item()
            predictions.append(out.view(-1).tolist())
            real_values = data.y.view(-1).tolist()
            if current_index >= starting_index:
                for real, pred in zip(real_values, out.view(-1).tolist()):
                    if abs(pred - real) <= tolerance * abs(real):
                        within_tolerance_count += 1
                    total_predictions += 1
            current_index += 1

    average_loss = total_loss / len(test_data_list)
    within_tolerance_percentage = (within_tolerance_count / total_predictions) * 100
    return average_loss, within_tolerance_percentage, predictions



In [ ]:
def plot_predictions(predictions,real_values=None, node = None):
    num_nodes = len(predictions[0])

    random_node = random.randint(0, num_nodes - 1) if node is None else node

    node_predictions = [p[random_node] for p in predictions]

    avg_predictions = [sum(p) / num_nodes for p in predictions]

    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(node_predictions, label='Predicted', marker='o')
    if real_values:
        node_real_values = [r[random_node] for r in real_values]
        plt.plot(node_real_values, label='Real', marker='x')
    plt.title(f'Predictions for Node {random_node}')
    plt.xlabel('Timestep')
    plt.ylabel('Bitrate')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(avg_predictions, label='Average Predicted', marker='o')
    if real_values:
        avg_real_values = [sum(r) / num_nodes for r in real_values]
        plt.plot(avg_real_values, label='Average Real', marker='x')
    plt.title('Average Predictions Across All Nodes')
    plt.xlabel('Timestep')
    plt.ylabel('Average Bitrate')
    plt.legend()

    plt.tight_layout()
    plt.show()



In [ ]:
def check_for_streaks(predictions, real_values, tolerance=0.15):
    real_values = real_values[1000:1050]
    predictions = predictions[1000:1050]
    streak_info = {}
    num_nodes = len(predictions[0])
    num_streaks = 0

    for node in range(num_nodes):
        streak = 0
        for timestep in range(len(predictions)):
            pred = predictions[timestep][node]
            real = real_values[timestep][node]
            if abs(pred - real) <= tolerance * abs(real):
                streak += 1
                if streak == 5:
                    streak_info[node] = timestep - 4
                    num_streaks += 1
                    break
            else:
                streak = 0

    return streak_info, num_streaks

# Testowanie i metryki dla wszystkich testów

In [ ]:
def test_all_sets_with_extended_run(model, all_test_data, name_of_test, tolerance=0.15):
    all_results = []
    test_restults_all = []
    all_streak_timesteps = []

    for test_data_list in all_test_data:
        streak_nodes = set()
        streak_timesteps = []
        within_tolerance = []

        average_loss, within_tolerance_percentage, predictions = test_model_with_tolerance(model, test_data_list)
        within_tolerance.append(within_tolerance_percentage)
        real_values = [data.y.view(-1).tolist() for data in test_data_list]
        
        streak_info, num_streaks = check_for_streaks(predictions, real_values, tolerance)
        streak_nodes = set(streak_info.keys())
        streak_timesteps = list(streak_info.values())
        
        rmse_per_timestep = []
        mape_per_timestep = []
        for timestep in range(1000, 1050):
            rmse_node = []
            mape_node = []
            for node in range(len(predictions[0])):
                rmse_node.append(root_mean_squared_error([real_values[timestep][node]], [predictions[timestep][node]]))
                mape_node.append(mean_absolute_percentage_error([real_values[timestep][node]], [predictions[timestep][node]]))
            rmse_per_timestep.append(rmse_node)
            mape_per_timestep.append(mape_node)

    
        num_streak_nodes = len(streak_nodes)
        avg_streak_timestep = np.mean(streak_timesteps) if streak_timesteps else None
        percentage_within_tolerance = np.mean(within_tolerance)
        percentage_nodes_with_streak = (num_streak_nodes / len(predictions[0])) * 100

        results_per_test = {
            "number_of_streak_nodes": num_streak_nodes,
            "average_streak_timestep": avg_streak_timestep,
            "percentage_within_tolerance": percentage_within_tolerance,
            "percentage_nodes_with_streak": percentage_nodes_with_streak,
            "rmse_per_timestep": rmse_per_timestep,
            "mape_per_timestep": mape_per_timestep
        }
        preds_and_real = {
            "predictions": predictions,
            "real_values": real_values
        }
        test_restults_all.append(results_per_test)
        all_results.append(preds_and_real)
        all_streak_timesteps.extend(streak_timesteps)

    with open(name_of_test +'_results.json', 'w') as json_file:
        json.dump(all_results, json_file, indent=4)

    overall_percentage_within_tolerance = np.mean([result["percentage_within_tolerance"] for result in test_restults_all])
    overall_avg_rmse_per_timestep = np.mean([result["rmse_per_timestep"] for result in test_restults_all], axis=0)
    overall_avg_mape_per_timestep = np.mean([result["mape_per_timestep"] for result in test_restults_all], axis=0)
    overall_percentage_nodes_with_streak = np.mean([result["percentage_nodes_with_streak"] for result in test_restults_all])
    avg_streak_timestep = np.mean(all_streak_timesteps) if all_streak_timesteps else None

    print(f"Overall Percentage of Nodes with Streak: {overall_percentage_nodes_with_streak:.2f}%")
    print(f"Overall Percentage of Predictions within Tolerance: {overall_percentage_within_tolerance:.2f}%")
    print(f"Overall Average Timestep at which Streak Happened: {avg_streak_timestep:.2f}")


    return all_results

def plot_average_metrics(overall_avg_rmse_per_timestep, overall_avg_mape_per_timestep):
    timesteps = range(1000, 1050) 
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    plt.plot(timesteps, overall_avg_rmse_per_timestep, label='Average RMSE', marker='o')
    plt.title('Average RMSE per Timestep')
    plt.xlabel('Timestep')
    plt.ylabel('RMSE')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(timesteps, overall_avg_mape_per_timestep, label='Average MAPE', marker='o')
    plt.title('Average MAPE per Timestep')
    plt.xlabel('Timestep')
    plt.ylabel('MAPE')
    plt.legend()

    plt.tight_layout()
    plt.show()

def calculate_metrics_from_json(json_file_path, tolerance=0.15):
    with open(json_file_path, 'r') as json_file:
        all_results = json.load(json_file)

    all_metrics = []
    all_streak_timesteps = []

    for test_result in all_results:
        predictions = test_result["predictions"]
        real_values = test_result["real_values"]

        total_within_tolerance = 0
        total_predictions = 0
        streak_nodes = set()
        streak_timesteps = []
        rmse_per_timestep = []
        mape_per_timestep = []

        for real_list, pred_list in zip(real_values, predictions):
            for real, pred in zip(real_list, pred_list):
                if abs(pred - real) <= tolerance * abs(real):
                    total_within_tolerance += 1
                total_predictions += 1

        streak_info, num_streaks = check_for_streaks(predictions, real_values, tolerance)
        streak_nodes = set(streak_info.keys())
        streak_timesteps = list(streak_info.values())

        rmse_per_timestep = []
        mape_per_timestep = []
        for timestep in range(1000, 1050):
            rmse_timestep = []
            mape_timestep = []
            for node in range(len(predictions[0])):
                rmse_timestep.append(root_mean_squared_error([real_values[timestep][node]], [predictions[timestep][node]]))
                mape_timestep.append(mean_absolute_percentage_error([real_values[timestep][node]], [predictions[timestep][node]]))
            rmse_per_timestep.append(np.mean(rmse_timestep))
            mape_per_timestep.append(np.mean(mape_timestep))

        num_streak_nodes = len(streak_nodes)
        avg_streak_timestep = np.mean(streak_timesteps) if streak_timesteps else None
        percentage_within_tolerance = (total_within_tolerance / total_predictions) * 100
        percentage_nodes_with_streak = (num_streak_nodes / len(predictions[0])) * 100

        test_metrics = {
            "number_of_streak_nodes": num_streak_nodes,
            "average_streak_timestep": avg_streak_timestep,
            "percentage_within_tolerance": percentage_within_tolerance,
            "percentage_nodes_with_streak": percentage_nodes_with_streak,
            "rmse_per_timestep": rmse_per_timestep,
            "mape_per_timestep": mape_per_timestep
        }
        all_metrics.append(test_metrics)
        all_streak_timesteps.extend(streak_timesteps)

    overall_percentage_within_tolerance = np.mean([metric["percentage_within_tolerance"] for metric in all_metrics])
    overall_avg_rmse_per_timestep = np.mean([metric["rmse_per_timestep"] for metric in all_metrics], axis=0)
    overall_avg_mape_per_timestep = np.mean([metric["mape_per_timestep"] for metric in all_metrics], axis=0)
    overall_percentage_nodes_with_streak = np.mean([metric["percentage_nodes_with_streak"] for metric in all_metrics])
    avg_streak_timestep = np.mean(all_streak_timesteps) if all_streak_timesteps else None

    print(f"Overall Percentage of Nodes with Streak: {overall_percentage_nodes_with_streak:.2f}%")
    print(f"Overall Percentage of Predictions within Tolerance: {overall_percentage_within_tolerance:.2f}%")
    print(f"Overall Average Timestep at which Streak Happened: {avg_streak_timestep:.2f}")

    return all_metrics, overall_avg_rmse_per_timestep, overall_avg_mape_per_timestep


# UCZENIE

In [10]:
def train(model, train_loader, optimizer, criterion):
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        h = torch.zeros(1, data.x.size(0), model.gru.hidden_size).to(data.x.device)
        out, h = model(data, h)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

# ZMIENNIE INICJALIZACJA

In [ ]:
data_folder = 'Zbiory danych/polaczenie_bitrate_7000/normalized/processed_7000_0'  
file_path = 'Zbiory danych/matrix.net'  
root_folder = "Zbiory danych/polaczenie_bitrate_7000/normalized"
adjacency_matrix = load_adjacency_matrix(file_path)



name_of_test = "GNN-Podstawowy"


graph_data_list = load_data_with_targets(data_folder, adjacency_matrix)
train_data = graph_data_list[:6000]
val_data = graph_data_list[6000:]


train_loader = DataLoader(train_data, batch_size=1, shuffle=False)

model = GNNModel(input_dim=1, hidden_dim=100, output_dim=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

# Wczytanie wszystkich danych do ewaluacji

In [ ]:
all_tests = get_all_test_data(root_folder, adjacency_matrix)

# Uczenie i ewaluacja modelu

In [ ]:
train(model, train_loader, optimizer, criterion)
torch.save(model.state_dict(), name_of_test + ".pth")
test_data = all_tests[0]
average_loss, within_tolerance_percentage, predictions = test_model_with_tolerance(model, test_data)
print(f'Average Loss: {average_loss}')
print(f'Percentage of Predictions within Tolerance: {within_tolerance_percentage:.2f}%')

real_values = [data.y.view(-1).tolist() for data in test_data]


In [ ]:
streak_info, num_streaks = check_for_streaks(predictions, real_values)
print(streak_info)
print(num_streaks)

In [ ]:
plot_predictions(predictions[1000:1050], real_values[1000:1050], node=40)

In [ ]:
result = test_all_sets_with_extended_run(model, all_tests, name_of_test)

In [ ]:
json_file_path = name_of_test + '_results.json'
metrics, overall_avg_rmse_per_timestep, overall_avg_mape_per_timestep = calculate_metrics_from_json(json_file_path)
plot_average_metrics(overall_avg_rmse_per_timestep, overall_avg_mape_per_timestep)